In [2]:
from tensorflow.keras.layers import Dense, LSTM


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor  # Correct import for XGBRegressor
from statsmodels.tsa.arima.model import ARIMA  # Updated import for ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

<h3>Actual Preprocessing</h3>

In [18]:
# Load the dataset from a CSV file
# Assuming the CSV file is named 'sales_data.csv'
training_data = pd.read_csv(r'C:\Users\MSI 11uc\Documents\ML\CW\Q5_sales_forecasting\data\raw\training_data.csv')
test_data = pd.read_csv(r'C:\Users\MSI 11uc\Documents\ML\CW\Q5_sales_forecasting\data\raw\test_data.csv')

In [28]:
# Check for duplicate rows
duplicate_rows = training_data[training_data.duplicated()]

# Print the duplicate rows, if any
print(f"Number of rows: {training_data.shape[0]}")
print(f"Number of duplicate rows: {duplicate_rows.shape[0]}")

Number of rows: 614098
Number of duplicate rows: 1766


In [29]:
# Drop duplicate rows
training_data = training_data.drop_duplicates()

In [30]:
# Check for missing values
print(training_data.isnull().sum())

date_id            0
item_dept          0
item_qty           0
net_sales          0
store              0
item               0
invoice_num    21427
day_of_week        0
dtype: int64


In [33]:
# Removing the outliers
training_data = training_data[training_data['net_sales'] < 300000]
training_data = training_data[training_data['item_qty'] < 800]

print(f"Number of rows: {training_data.shape[0]}")


Number of rows: 612330


In [35]:
### Groubpy the dataset based on date, net sales, item quantity
grouped_df = training_data.groupby(['date_id', 'store', 'item_dept']).agg({
    'item_qty': 'sum',
    'net_sales': 'sum'
}).reset_index()

# Display the first few rows of the grouped dataframe
print(grouped_df.head())

     date_id store  item_dept  item_qty     net_sales
0 2021-11-01   ABC  Beverages   908.000  254007.79901
1 2021-11-01   ABC    Grocery  2360.399  437286.62402
2 2021-11-01   ABC  Household  1043.000  247336.26601
3 2021-11-01   XYZ  Beverages   779.000  187931.76902
4 2021-11-01   XYZ    Grocery  2974.060  489967.72300


In [ ]:
# Ensure that each row is uniquely identified by 'store', 'item_dept', and 'date_id'
grouped_df['primary_key'] = grouped_df['store'] + '|' + grouped_df['item_dept'] + '|' + grouped_df['date_id'].astype(str)

# Check for duplicates based on primary keys
if grouped_df.duplicated(subset=['primary_key']).any():
    print("Warning: There are duplicate rows based on the primary keys.")
else:
    print("All rows are uniquely identified by the primary keys.")

In [ ]:
# The target variable is the daily sales quantity for each store and department
target_variable = grouped_df[['store', 'item_dept', 'date_id', 'item_qty']]

# Rename the target variable column for clarity
target_variable.rename(columns={'item_qty': 'daily_sales_qty'}, inplace=True)

# Display the target variable
print(target_variable.head())